In [2]:
import os

We will use `fastText` pretrained word vectors (Mikolov et al., 2017), trained on 600 billion tokens on Common Crawl. fastText is an upgraded version of word2vec and outperforms other state-of-the-art methods by a large margin.

In [3]:
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2023-11-04 04:33:37--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.108, 3.163.189.14, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z   6%[>                   ]  87.87M   146MB/s               

crawl-300d-2M.vec.z 100%[===================>]   1.42G   186MB/s    in 7.3s    

2023-11-04 04:33:44 (199 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  
CPU times: user 373 ms, sys: 119 ms, total: 492 ms
Wall time: 43.8 s


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Data preparation

To prepare our text data for training, first we need to tokenize our sentences and build a vocabulary dictionary `word2idx`, which will later be used to convert our tokens into indexes and build an embedding layer.

An embedding layer serves as a look-up table which takes words’ indexes in the vocabulary as input and output word vectors. Hence, the embedding layer has shape `\((N, d)\)` where `\(N\)` is the size of the vocabulary and `\(d\)` is the embedding dimension. In order to fine-tune pretrained word vectors, we need to create an embedding layer in our nn.Module class. Our input to the model will then be `input_ids`, which is tokens’ indexes in the vocabulary.

In [6]:
import pandas as pd
import numpy as np
import ast


def clean_steps(data):
    data_list = ast.literal_eval(data)
    return ', '.join(data_list)


BOS, EOS = ' ', '\n'
data = pd.read_csv('RAW_recipes.csv')
data = data[['name', 'steps', 'description', 'ingredients']]
data = data[~data['steps'].str.contains('please ignore')]
data = data[~data['steps'].str.contains('none')]
data['steps'] = data['steps'].apply(clean_steps)
data['ingredients'] = data['ingredients'].apply(clean_steps)

lines = data.apply(lambda row: str(row['name']) + 
                   '; ' + 
                   str(row['steps']).replace("\n", ' ') + 
                   '; ' + 
                   str(row['description']) + 
                   '; ' + 
                   str(row['ingredients'])[:512], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()